In [1]:
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout

Using TensorFlow backend.


In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
classifier = Sequential()

# Paso 1 - Convolución
classifier.add(Conv2D(filters = 64,kernel_size = (3, 3), 
                      input_shape = (64, 64, 3), activation = "relu"))

# Paso 2 - Max Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.2))

# Una segunda capa de convolución y max pooling
classifier.add(Conv2D(filters = 128,kernel_size = (3, 3), activation = "relu"))

classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.2))

# Una segunda capa de convolución y max pooling
classifier.add(Conv2D(filters = 32,kernel_size = (3, 3), activation = "relu"))

classifier.add(MaxPooling2D(pool_size = (2,2)))
classifier.add(Dropout(0.2))

# Paso 3 - Flattening
classifier.add(Flatten())

# Paso 4 - Full Connection
classifier.add(Dense(units = 128, activation = "relu"))
classifier.add(Dropout(0.2))
classifier.add(Dense(units = 64, activation = "relu"))
classifier.add(Dense(units = 4, activation = "sigmoid"))

# Compilar la CNN
classifier.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

classifier.summary()

# Parte 2 - Ajustar la CNN a las imágenes para entrenar 



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 62, 62, 64)        1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 12, 12, 32)       

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_dataset = train_datagen.flow_from_directory('./selfie-project/dataset_gestos/train/',
                                                    target_size=(64, 64),
                                                    batch_size=32,
                                                    class_mode='binary')

testing_dataset = test_datagen.flow_from_directory('./selfie-project/dataset_gestos/test/',
                                                target_size=(64, 64),
                                                batch_size=32,
                                                class_mode='binary')

classifier.fit_generator(training_dataset,
                        steps_per_epoch=1596,
                        epochs=3,
                        validation_data=testing_dataset,
                        validation_steps=405)

Found 2864 images belonging to 4 classes.
Found 1085 images belonging to 4 classes.
Epoch 1/3
1596/1596 [==============================] - 454s 285ms/step - loss: 0.2668 - accuracy: 0.8879 - val_loss: 2.8778 - val_accuracy: 0.7088
Epoch 2/3
1596/1596 [==============================] - 440s 276ms/step - loss: 0.0349 - accuracy: 0.9887 - val_loss: 4.1577 - val_accuracy: 0.7211
Epoch 3/3
1596/1596 [==============================] - 441s 277ms/step - loss: 0.0191 - accuracy: 0.9935 - val_loss: 3.7581 - val_accuracy: 0.7137


In [5]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('./selfie-project/dataset_gestos/test/mano_cerrado/aabdtvka.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = test_image / 255.0
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image) 
print(result)

[[5.0067902e-06 1.1920929e-07 9.6025413e-01 0.0000000e+00]]


In [6]:
classifier.save("gestos.h5")

In [7]:
training_dataset.class_indices

{'desconocido': 0, 'mano_abierta': 1, 'mano_cerrado': 2, 'mano_dedo': 3}